import required packages

In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'sklearn'

Initialize image data generator with rescaling

In [3]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

Preprocess all test images

In [4]:
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


Preprocess all train images

In [5]:
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


create model structure

In [6]:
emotion_model = Sequential()

In [7]:
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [8]:
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [9]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [10]:
cv2.ocl.setUseOpenCL(False)

In [11]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

c:\Users\jkjhkl\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Train the neural network/model

In [12]:
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=2870 // 64,
        epochs=7,
        validation_data=validation_generator,
        validation_steps=717 // 64)

C:\Users\jkjhkl\AppData\Local\Temp\ipykernel_6692\3847014661.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/7
44/44 [==============================] - 23s 501ms/step - loss: 1.8466 - accuracy: 0.2461 - val_loss: 1.8509 - val_accuracy: 0.2401
Epoch 2/7
44/44 [==============================] - 21s 483ms/step - loss: 1.8315 - accuracy: 0.2337 - val_loss: 1.8422 - val_accuracy: 0.2472
Epoch 3/7
44/44 [==============================] - 21s 472ms/step - loss: 1.8346 - accuracy: 0.2436 - val_loss: 1.8404 - val_accuracy: 0.2557
Epoch 4/7
44/44 [==============================] - 21s 469ms/step - loss: 1.8197 - accuracy: 0.2574 - val_loss: 1.8206 - val_accuracy: 0.2756
Epoch 5/7
44/44 [==============================] - 22s 505ms/step - loss: 1.8097 - accuracy: 0.2514 - val_loss: 1.8327 - val_accuracy: 0.2401
Epoch 6/7
44/44 [==============================] - 19s 425ms/step - loss: 1.8126 - accuracy: 0.2521 - val_loss: 1.8201 - val_accuracy: 0.2642
Epoch 7/7
44/44 [==============================] - 18s 409ms/step - loss: 1.8077 - accuracy: 0.2507 - val_loss: 1.8245 - val_accuracy: 0.2727


save model structure in jason file

In [ ]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

save trained model weight in .h5 file

In [ ]:
emotion_model.save_weights('emotion_model.h5')
#test
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

load json and create model

In [ ]:
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

load weights into new model

In [ ]:
emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")

start the webcam feed<br>
ap = cv2.VideoCapture(0)

pass here your video path<br>
you may download one from here : https://www.pexels.com/video/three-girls-laughing-5273028/

In [ ]:
cap = cv2.VideoCapture("C:\\JustDoIt\\ML\\Sample_videos\\emotion_sample6.mp4")

In [ ]:
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

load json and create model

In [ ]:
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

load weights into new model

In [13]:
emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


Initialize image data generator with rescaling

In [14]:
test_data_gen = ImageDataGenerator(rescale=1./255)

Preprocess all test images

In [15]:
test_generator = test_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


do prediction on test data

In [16]:
from keras.models import load_model
model = load_model("model/emotion_model.h5")

C:\Users\jkjhkl\AppData\Local\Temp\ipykernel_6692\2654746365.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = emotion_model.predict_generator(test_generator)


see predictions<br>
for result in predictions:<br>
    max_index = int(np.argmax(result))<br>
    print(emotion_dict[max_index])

In [17]:
print("-----------------------------------------------------------------")
# confusion matrix
c_matrix = confusion_matrix(test_generator.classes, predictions.argmax(axis=1))
print(c_matrix)
cm_display = ConfusionMatrixDisplay(confusion_matrix=c_matrix, display_labels=emotion_dict)
cm_display.plot(cmap=plt.cm.Blues)
plt.show()

-----------------------------------------------------------------


NameError: name 'confusion_matrix' is not defined

Classification report

In [ ]:
print("-----------------------------------------------------------------")
print(classification_report(test_generator.classes, predictions.argmax(axis=1)))